## Step1: Convert images in to Tensorflow record

input: the folder of images to be detected, the destination path, and a identifier prefix

output: a TFrecord of the images



note that, images in the folder should be consecutive frames of a infrared video

naming convention must be prefix_frame#.jpg 
    
prefix must be letters ONLY, frame# is auto-generated by ResearchIR

the code use frame# to sort the order of the frames
    
SB_1234.jpg is vaild

2018SB_1234.jpg is invalid

## Imports

In [1]:
from tkinter import *
import tkinter.filedialog
import tkinter.messagebox
import subprocess
import os
import re
import tensorflow as tf
import numpy as np
import time
import threading
from PIL import Image



## Variables
Here you can set the default values to save time

remember to replace the 'not selected' strings in GUI at the bottom

In [2]:
images_dir = ''
destination = ''
output_prefix = ''
image_record = ''
inference_record = ''
inference_graph = ''
script_dir = 'inference/infer_detections.py'
converted=False

args = ['python', script_dir, '--input_tfrecord_paths=', '--output_tfrecord_path=', '--inference_graph=']

## Functions

This notebook should have been Converison_and_Inference instead of just Conversion

However, the inference subprocess always encounter segmentation fault... Now it's seperated in another notebook

In [3]:
def display(string):
     Gui.t.insert(END,string +'\n')

def askdestination():
    global destination
    destination = tkinter.filedialog.askdirectory()
    Gui.entry_3.configure(text=destination)

def askgraph():
    global inference_graph
    inference_graph = tkinter.filedialog.askopenfilename()
    Gui.entry_2.configure(text=inference_graph)

def askdirectory():
    global images_dir
    images_dir = tkinter.filedialog.askdirectory()
    Gui.entry_1.configure(text=images_dir)

def infer():
    p = subprocess.Popen(args,shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    display('Please wait...')
    p.wait()
    """while p.poll()==None:
        out, err = p.communicate()
        out = out.decode("ascii",errors="ignore")
        err = err.decode("ascii",errors="ignore")
        for line in out.splitlines():
            display(line)
        for line in err.splitlines():
            display(line)
        GUI.after(1000)
    """
    
    out, err = p.communicate()
    out = out.decode("ascii",errors="ignore")
    err = err.decode("ascii",errors="ignore")
    for line in out.splitlines():
        display(line)
    for line in err.splitlines():
        display(line)
        GUI.after(500)
    if p.returncode==0:
        tkinter.messagebox.showinfo('Success','Inference TFrecord is created!')
    else:
        tkinter.messagebox.showerror('Error: '+str(p.returncode),'TFrecord is not created. Please check.')
        print(p.returncode)
    p.terminate()
    
def convert_infer():
    display('Start converting images to TFrecord...')
    global image_record
    image_record = output_prefix+'_images.record'
    f = open(destination+'/fileQueue.txt', 'w')
    writer1 = tf.python_io.TFRecordWriter(os.path.join(destination,image_record))
    #sort in arthmetric order
    sorted_list = sorted(os.listdir(images_dir),key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    jump=0
    for idx,img in enumerate(sorted_list):
        if not img.endswith('.jpg'):
            jump=jump+1
            continue
        if (idx-jump)%1000==0 and idx>0:
            display(str(idx)+' images converted')
            #print(str(idx)+' images Processed')
        b = img.encode()
        f.writelines(img+'\n')
        this_path = images_dir+'/'+img
        fid = tf.gfile.GFile(this_path, 'rb')
        encoded_jpg = fid.read()
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    "label": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img.encode('utf8')])),
                    "image/encoded": tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_jpg]))}
            )
        )
        writer1.write(example.SerializeToString())
    writer1.close()
    f.close()
    time.sleep(2)
    #tkinter.messagebox.showinfo('Conversion Complete','All images are converted into TFrecord')
    display('All images are converted into TFrecord')
    global converted
    converted = True

    """
    display('Start inferencing TFrecord...')
    print('Start inferencing TFrecord...')
    global inference_record
    global args
    inference_record = output_prefix+'_inference.record'
    args[2]='--input_tfrecord_paths='+image_record
    args[3]='--output_tfrecord_path='+destination+'/'+inference_record
    args[4]='--inference_graph='+inference_graph
    infer()
    #threading.Thread(target=infer).start()
    
    p = subprocess.Popen(args,shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    p.wait()
    out, err = p.communicate()
    out = out.decode("ascii",errors="ignore")
    err = err.decode("ascii",errors="ignore")
    for line in out.splitlines():
        display(line)
    for line in err.splitlines():
        display(line)
    time.sleep(1)
    if p.returncode==0:
        tkinter.messagebox.showinfo('Success','Inference TFrecord is created!')
    else:
        tkinter.messagebox.showerror('Error','TFrecord is not created. Please check.')
    p.terminate()
    """
def convert_infer_button():
    Gui.t.delete(1.0,END)
    global output_prefix
    output_prefix = Gui.entry_4.get()
    if images_dir==''or destination=='':
        tkinter.messagebox.showwarning('Alert','Please select source and destination directory.')
    #elif not inference_graph.endswith('.pb'):
    #    tkinter.messagebox.showwarning('Alert','Please select model.')
    elif output_prefix == '':
        tkinter.messagebox.showwarning('Alert','Please specify prefix for output. ')
    else:
        threading.Thread(target=convert_infer).start()
        #convert_infer()

## User Interface

In [4]:
class Gui:
    root = Tk()
    root.minsize(480,320)
    root.title('Convert images to TFrecord')
    
    label_1=Label(root,text = 'Images folder: ')
    #label_2=Label(root,text = 'Inference model: ')
    label_3=Label(root,text = 'Output directory: ')
    label_4=Label(root,text = 'Output prefix: ')
    entry_1=Label(root,text = 'not selected',bg='White')
    #entry_2=Label(root,text = 'not selected',bg='White')
    entry_3=Label(root,text = 'not selected',bg='White')
    entry_4=Entry(root,bg = 'White')
    button_1=Button(root,text='Browse',command= askdirectory)
    #button_2=Button(root,text='Browse',command= askgraph)
    button_3=Button(root,text='Browse',command= askdestination)
    button_4=Button(root,text='convert',command= convert_infer_button)

    label_1.grid(row=0,sticky=E)
    #label_2.grid(row=1,sticky=E)
    label_3.grid(row=2,sticky=E)
    label_4.grid(row=3,sticky=E)
    entry_1.grid(row=0,column=1)
    #entry_2.grid(row=1,column=1)
    entry_3.grid(row=2,column=1)
    entry_4.grid(row=3,column=1)
    button_1.grid(row=0,column=2)
    #button_2.grid(row=1,column=2)
    button_3.grid(row=2,column=2)
    button_4.grid(row=3,column=2)
    t = Text(root,bg='White')
    t.grid(row=5,column=0,columnspan=3)
    
GUI = Gui.root
GUI.mainloop()